In [44]:
# Machine Learning ile Takeoff Weight hesaplanmaktadır. 
# Uçuşun Climb fazındaki featureler kullanılarak model oluşturulmuştur. 

import pandas as pd
from aircraftLib import JetAircraft
from UtilityFunctions import *
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import os
#from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import style
import time

from sklearn import model_selection 
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, SGDRegressor,BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from xgboost.sklearn import XGBRegressor



from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from tsfresh.feature_extraction import feature_calculators, settings
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from tsfresh import extract_features, extract_relevant_features, select_features

from tpot import TPOTRegressor


#from tsfresh.utilities.distribution import MultiprocessingDistributor
#from tsfresh.utilities.dataframe_functions import impute_dataframe_range
#from sklearn.metrics import accuracy_score


In [ ]:
style.use('ggplot')

basepath = "C:/Users/Recep/Desktop/Boeing Proje/"  ## Configure this line
odp = 'B777_Data'  ## Airport Pairs
datapath = basepath + odp

files = [f for f in listdir(datapath) if isfile(join(datapath, f))]
fig1 = plt.figure(1,figsize=[7, 5])
ac = JetAircraft('B773ERGE115B')

constants = {}
constants["lb_to_kg"] = 0.45359
constants["hr_to_sec"] = 1/3600
constants["m_to_nm"] = 0.0005399568
constants["knot_to_ms"] = 0.51444
constants["ft_to_m"] = 0.3048 

knot_to_ms = constants["knot_to_ms"]
ft_to_m = constants["ft_to_m"]
g0 = 9.8065
g0_feet = 32.1741

In [ ]:
from cmath import nan

counter = 0
y = []
specificEnergyGradient = []
specificEnergy = []
flightPathAngle = []
distance_from_takeoff_list = []
landing_gear = []


df = pd.DataFrame()
for file in files:
    counter +=1
    """ if counter == 4:
        break """
    filepath = os.path.join(odp, file)
    data = pd.read_csv(filepath, low_memory=False, skiprows=1)
    data.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
    data['id'] = np.full((len(data),1), counter-1)
    mass_0 = data.aZFW[0] * constants['lb_to_kg'] + data.aTFQ[0]
    fuel_flow = (data.aFF1+data.aFF2)*constants["lb_to_kg"]*constants["hr_to_sec"]
    

    data.drop(columns=['aGS','TRUTRAKANG','aTHRANG1','aTHRANG2','aFLAP','aGW','aWNDDIR','aWNDSPD','aSPDBRK','TRUEHEADNG','aFF1','aFF2','aTFQ','aZFW','WFCMD'], axis=1,inplace=True)
    
    y.append(mass_0)
    #df = df.append(data, ignore_index=True)

    distance_from_takeoff = 0
    fuel_consumption = 0 
    mass = []
    takeoff_alt_list = []
    takeoff_tas_list = []
    sigma = pressureRatio(data.aSAP)
    theta = temperatureRatio(data.aSAT)
    isaDev = isaDeviation(data.aALTBARO, data.aSAT)
    tas = cas2tas(data.aCAS, data.aALTBARO, isaDev)
    j=0

    for i in range(len(data)-1):
        
        if data.aLDGDSEL[i] == "DOWN":
            landing_gear.append(1)
        else:
            landing_gear.append(0)
        
        horizontalSpeed = haversine((data.LATPOS[i],data.LONPOS[i]),(data.LATPOS[i+1],data.LONPOS[i+1]))
        distance_from_takeoff = distance_from_takeoff + haversine((data.LATPOS[i],data.LONPOS[i]),(data.LATPOS[i+1],data.LONPOS[i+1])) * constants["m_to_nm"]
        flightMode, ROCD = ac.identifyFlightMode(data.aALTBARO[i+1],data.aALTBARO[i])

        fuel_consumption += fuel_flow[i]
        mass.append(mass_0 - fuel_consumption)         

        #drag = ac.calculate_drag(mass[i],sigma[i],data.aMACH[i],0)
        j+=1
        #if np.abs(distance_from_takeoff) > 5:
        if data.aALTBARO[i] > 1500:
            j-=1
            
            Vdot = (tas[i+1]-tas[i]) * ft_to_m
            distance_from_takeoff_list.append(distance_from_takeoff)
            flightPathAngle.append(np.arctan2(ROCD,horizontalSpeed))
            specEnergyGrad = tas[i] * ft_to_m * Vdot + g0 * ROCD * ft_to_m
            specificEnergyGradient.append(specEnergyGrad)
            specificEnergy.append(0.5*((tas[i]*ft_to_m)**2) + g0 * data.aALTBARO[i] * ft_to_m)    
            #if np.abs(distance_from_takeoff) > 50:    # Distance to Takeoff
            #df = df.append(data.iloc[j:i+1], ignore_index=True)
                #break
            #if distance_from_takeoff > 10:
        if data.aALTBARO[i] > 25000: 
            df = df.append(data.iloc[j:i+1], ignore_index=True)
            break
        

#new_df = df[['Time','aALTBARO','aCAS','id']].copy()
#new_df['specEnergyGrad'] = pd.Series(specificEnergyGradient)
df.drop(columns=['aMACH','aLDGDSEL','LATPOS','LONPOS','aSAP','aSAT'], axis=1,inplace=True)
df['specEnergyGrad'] = pd.Series(specificEnergyGradient)
#df['aLDGDSEL'] = pd.Series(landing_gear)
#df['specEnergy'] = pd.Series(specificEnergy)
df['flightPathAngle'] = pd.Series(flightPathAngle)
df['dist2takeoff'] = pd.Series(distance_from_takeoff_list)
df['aALTBARO'] = df['aALTBARO'] * ft_to_m
df['aCAS'] = df['aCAS'] * knot_to_ms
# df['aGS'] = df['aGS'] * knot_to_ms
# df['aTHRANG1'] = df['aTHRANG1'] / 100
# df['aTHRANG2'] = df['aTHRANG2'] / 100

labels = pd.Series(y)
df.head()

In [ ]:
## Kaydedilmiş data kullanılacaksa çalıştırma burayı
extraction_settings = MinimalFCParameters() 
#extraction_settings = ComprehensiveFCParameters() 
#extraction_settings = EfficientFCParameters() 

# features_filtered_direct = extract_relevant_features(df, labels,
#                                                      column_id='id', column_sort='Time')
X = extract_features(df, column_id='id', column_sort='Time',
                     default_fc_parameters=extraction_settings,
                     # we impute = remove all NaN features automatically
                     impute_function=impute)

X.head()

In [ ]:
## Kaydedilen değişkenleri yüklemek için burayı çalıştır

%store -r X_filtered
%store -r X
%store -r labels
%store -r df

In [ ]:
""" prev_item = 0
for item in df['id']:
    if item - prev_item > 1:
        print(item)
        break
    else:
        prev_item = item

print(df['id'] == 372) """

new_labels = labels.drop(372) ### X datasında 372. data alınmamış bozuk

    

In [ ]:
X_filtered = select_features(X, new_labels)
X_filtered.head()

In [ ]:
scaler = StandardScaler()
scaler.fit(X_filtered)
scaled_data = scaler.transform(X_filtered)

In [ ]:
pca = PCA(n_components=5)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
x_pca

In [ ]:
### TPOT Regression
# X_full_train, X_full_test, y_train, y_test = train_test_split(X_filtered, new_labels, test_size=.25, train_size=.75)
# tpot = TPOTRegressor()
# tpot.fit(X_full_train, y_train)

In [ ]:
# print(tpot.score(X_full_test, y_test))
# tpot.export('tpot_b777.py')

In [74]:
X_full_train, X_full_test, y_train, y_test = train_test_split(x_pca, new_labels, test_size=.25)
regressor = ExtraTreesRegressor()
regressor.fit(X_full_train, y_train)
accuracy = regressor.score(X_full_test, y_test)
print(accuracy)


0.8973891877927579


In [ ]:
a = pd.Series(data=regressor.feature_importances_, index=X_filtered.columns).sort_values(ascending=False)
print(a[:30])

In [75]:
takeoff_weight_estimation = regressor.predict(X_full_test)
takeoff_weight_estimation_percentage = takeoff_weight_estimation / ac.reference_mass*100
takeoff_weight_percentage = y_test / ac.reference_mass*100
takeoff_weight_error = (y_test - takeoff_weight_estimation)/ac.reference_mass*100
mean_takeoff_weight_error = np.mean(np.abs(takeoff_weight_error))
std_takeoff_weight_error = np.std(takeoff_weight_error)

print("Actual Weights Mean: ", np.mean(y_test))
print("Actual Weights Standard Deviation: ", np.std(y_test))
print("Actual Weights Max: ", np.max(y_test))
print("Actual Weights Min: ", np.min(y_test))
print("Estimated Weights Mean: ", np.mean(takeoff_weight_estimation))
print("Estimated Weights Standard Deviation: ", np.std(takeoff_weight_estimation))
print("Estimated Weights Max: ", np.max(takeoff_weight_estimation))
print("Estimated Weights Min: ", np.min(takeoff_weight_estimation))

Actual Weights Mean:  309213.10926431353
Actual Weights Standard Deviation:  38935.69304513861
Actual Weights Max:  351449.46400000004
Actual Weights Min:  212960.9696
Estimated Weights Mean:  308533.48244875297
Estimated Weights Standard Deviation:  35823.822679785975
Estimated Weights Max:  344081.2735839997
Estimated Weights Min:  219863.34432


In [76]:
%matplotlib qt
fig1 = plt.figure(figsize=[7, 5])
x = range(0,100)
plt.plot(x,x, linewidth=.5, color = 'r')
plt.scatter(takeoff_weight_percentage, takeoff_weight_estimation_percentage, s = 2, color = 'k')
plt.title('Take-off Weight Estimation', fontsize=12, y=1.1)
#plt.xlim(50, 100)
#plt.ylim(50, 100)
plt.ylabel('MTOW(%) Estimated')
plt.xlabel('MTOW(%) Actual')
plt.grid()

fig2, ax2 = plt.subplots()
plt.hist(takeoff_weight_error, bins=26, edgecolor = "black")
plt.title('Take-off Weight Estimation Error', fontsize=12, y=1.1)
plt.ylabel('Number of Aircraft')
plt.xlabel('MTOW(%) Error')
props = dict(boxstyle='square', facecolor='cyan', alpha=0.5)
textstr =  f'Mean Absolute Error: {np.abs(mean_takeoff_weight_error):.2f}%MTOW\n Standart Deviation:{std_takeoff_weight_error:.2f}%MTOW'
plt.text(0.05, 0.95, textstr, transform = ax2.transAxes, fontsize = 8, verticalalignment ='top', bbox = props)
plt.xlim(-40, 40)


(-40.0, 40.0)

In [ ]:
fig3 = plt.figure(facecolor='#f0f0f0',figsize=[15,7])
ax1 = plt.subplot2grid((8,1), (0,0), rowspan=2, colspan=1)
plt.ylabel('CAS (m/s)')
plt.title('B777 Flight Data wrt Distance from Takeoff')
ax2 = plt.subplot2grid((8,1), (2,0), rowspan=2, colspan=1, sharex=ax1)
plt.ylabel('Altitude (m)')
ax2v = ax2.twinx()
ax3 = plt.subplot2grid((8,1), (4,0), rowspan=2, colspan=1, sharex=ax1)
plt.ylabel('Specific Energy Gradient')
ax4 = plt.subplot2grid((8,1), (6,0), rowspan=2, colspan=1, sharex=ax1)
plt.ylabel('Specific Energy Gradient')
plt.xlabel('Distance From Takeoff (NM)')

plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax3.get_xticklabels(), visible=False)

prev_index = 0
distance = []
alt = []
airspeed = []
energy_grad = []
flightpathang = []

j = 0
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
for index in df['id']:
    if index != prev_index:
        color_idx = index % len(colors)
        ax1.plot(distance, airspeed, linewidth=1, color=colors[color_idx], label=str(new_labels[index]))
        ax2.plot(distance, alt, linewidth=1, color=colors[color_idx], label=str(new_labels[index]))
        ax3.plot(distance, energy_grad, linewidth=1, color=colors[color_idx], label=str(new_labels[index]))
        ax4.plot(distance, flightpathang, linewidth=1, color=colors[color_idx], label=str(new_labels[index]))
        
        prev_index = index
        distance = []
        alt = []
        airspeed = []
        energy_grad = []
        flightpathang = []
        if index == 7:
            break
    distance.append(df['dist2takeoff'][j])
    alt.append(df['aALTBARO'][j])
    airspeed.append(df['aCAS'][j])
    energy_grad.append(df['specEnergyGrad'][j])
    flightpathang.append(df['flightPathAngle'][j])
    j+=1
    
plt.legend(loc=4, ncol=2)
plt.show()

In [ ]:
### İşlemler uzun sürüyor o yüzden önemli değişkenleri kaydediyoruz

%store X_filtered
%store labels 
%store X 
%store df